# Initilize Environment

## Initial Kaggle Imports
Commented out and will be deleted later.

In [9]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

#import numpy as np # linear algebra
#import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Specific Imports
imports used for the specific model tasks

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import timm

import pandas as pd
import numpy as np
import random

import sys
from tqdm.notebook import tqdm

# Dataset Setup

In [11]:
#Extremely simple data set class.
class OurDataSetA(Dataset):
    def __init__(self, data_directory, transform=None):
        self.data = ImageFolder(data_directory, transform=transform)

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, at_index):
        return self.data[at_index]

    @property
    def classes(self):
        return self.data.classes
#END CLASS

#Strings of data directories
str_data_dir_train = '/kaggle/input/balanced-raf-db-dataset-7575-grayscale/train'
str_data_dir_valid = '/kaggle/input/balanced-raf-db-dataset-7575-grayscale/val'
str_data_dir_test  = '/kaggle/input/balanced-raf-db-dataset-7575-grayscale/test'

#Transforms
transform_a = transforms.Compose([
    transforms.ToTensor()
])

# Order matters: spatial ops (resize/flip) → photometric (color) → tensor/normalize
transform_b = transforms.Compose([
    # Resize all images to a fixed size (H, W). Models expect consistent shapes.
    transforms.Resize((base_size)),

    # Randomly flip left-right with probability p.
    # p=0.5 means ~50% of images flipped; improves robustness to mirror variations.
    transforms.RandomHorizontalFlip(p=0.5),

    # Slight brightness/contrast jitter to reduce overfitting.
    # brightness=0.1 allows +/-10% brightness; contrast=0.1 allows +/-10% contrast.
    transforms.ColorJitter(brightness=0.1, contrast=0.1),

    # Convert PIL Image (H, W, C) in [0, 255] to torch.Tensor (C, H, W) in [0.0, 1.0].
    transforms.ToTensor(),

    # Optional: Uncomment if using pretrained backbones (e.g., ResNet/EfficientNet on ImageNet)
    # Scales channels to ImageNet mean/std: mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]
    # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

#Dataset variables
dataset_train = OurDataSetA(str_data_dir_train, transform_a)
dataset_valid = OurDataSetA(str_data_dir_valid, transform_a)
dataset_test  = OurDataSetA(str_data_dir_test, transform_a)

#Label Dictionary
label_dict ={
    0:"Angry",
    1:"Disgust",
    2:"Fear",
    3:"Happy",
    4:"Neutral",
    5:"Sad",
    6:"Surprise"
}

#This just serves to take a random snippet from the dataset and display it for demo purposes
var_rand = random.randint(1,(4289*7))
image, label = dataset_train[var_rand]
print("index used: ", var_rand)
print(label_dict[label])
image

index used:  14829
Happy


tensor([[[0.7059, 0.7255, 0.7490,  ..., 0.1216, 0.0863, 0.0549],
         [0.7059, 0.7216, 0.7451,  ..., 0.1451, 0.1098, 0.0824],
         [0.7020, 0.7216, 0.7451,  ..., 0.1804, 0.1451, 0.1176],
         ...,
         [0.0157, 0.2471, 0.5176,  ..., 0.0039, 0.0039, 0.0039],
         [0.0157, 0.1137, 0.2627,  ..., 0.0039, 0.0039, 0.0039],
         [0.0235, 0.0078, 0.0431,  ..., 0.0039, 0.0039, 0.0039]],

        [[0.7059, 0.7255, 0.7490,  ..., 0.1216, 0.0863, 0.0549],
         [0.7059, 0.7216, 0.7451,  ..., 0.1451, 0.1098, 0.0824],
         [0.7020, 0.7216, 0.7451,  ..., 0.1804, 0.1451, 0.1176],
         ...,
         [0.0157, 0.2471, 0.5176,  ..., 0.0039, 0.0039, 0.0039],
         [0.0157, 0.1137, 0.2627,  ..., 0.0039, 0.0039, 0.0039],
         [0.0235, 0.0078, 0.0431,  ..., 0.0039, 0.0039, 0.0039]],

        [[0.7059, 0.7255, 0.7490,  ..., 0.1216, 0.0863, 0.0549],
         [0.7059, 0.7216, 0.7451,  ..., 0.1451, 0.1098, 0.0824],
         [0.7020, 0.7216, 0.7451,  ..., 0.1804, 0.1451, 0.

# Data Loaders
loading... loading... loading...

In [12]:
#These just make use of the pre-made class DataLoader so there's no need to define our own here
loader_train = DataLoader(dataset_train, batch_size = 32, shuffle = True)
loader_valid = DataLoader(dataset_valid, batch_size = 32, shuffle = False)
loader_test  = DataLoader(dataset_test, batch_size = 32, shuffle = False)

# Classifier
the model itself for simple tasks of classification.

In [13]:
class EmotionClassifier(nn.Module):
    def __init__(self, num_classes=7):
        super().__init__()
        self.base_model = timm.create_model('efficientnet_b0',pretrained=True) #Set base model
        self.features = nn.Sequential(*list(self.base_model.children())[:-1])

        enet_out_size = 1280
        
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(enet_out_size, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        output = self.classifier(x)
        return output
#END CLASS

#Create the model, we'll call it model_one.
model_one = EmotionClassifier(num_classes=7)

#this is just done to show a snippet of the models layout.
print(str(model_one)[:300])
        

EmotionClassifier(
  (base_model): EfficientNet(
    (conv_stem): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): BatchNormAct2d(
      32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
      (drop): Identity()
      (act): SiLU(inplace=True)



# Training
Queue eye of the tiger

In [14]:
#Set up for the loop
# Loss Function
criterion = nn.CrossEntropyLoss()

# Optimizer
optimizer = optim.Adam(model_one.parameters(), lr=0.001)

# Length in Epochs
number_of_epochs = 5

# Losses Arrays
training_losses = []
validation_losses = []

# Establish Device settings
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Move model to device
model_one.to(device) #model_one is made in the Classifier segment
print('--') #Kaggle really really wants to put things so if I don't give it this it outputs the entire layout of the model below, and its a lot of text man.


--


In [ ]:
#The actual loop
for epoch in range(number_of_epochs):
    #Training Phase
    model_one.train() #Signal to the model that we're training
    running_loss = 0.0 #Current loss of the session
    for images, labels in tqdm(loader_train, desc='Training Phase'): #
        # Move images and labels to the device
        images = images.to(device)
        labels = labels.to(device)

        #
        optimizer.zero_grad()
        outputs = model_one(images)##
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * labels.size(0)
        
    # Loss Tracking
    train_loss = running_loss / len(loader_train.dataset)
    training_losses.append(train_loss)
    
    #Validation Phase
    model_one.eval() #Signal to the model that we're not training.
    running_loss = 0.0
    with torch.no_grad():
        for images, labels in tqdm(loader_valid, desc='Validation loop'):
            # Move inputs and labels to the device
            images = images.to(device)
            labels = labels.to(device)

            #
            outputs = model_one(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * labels.size(0)

    # Loss Tracking
    valid_loss = running_loss / len(loader_valid.dataset)
    validation_losses.append(valid_loss)
    print(f"Epoch {epoch+1}/{number_of_epochs} - Train loss: {train_loss}, Validation loss: {valid_loss}")

Training Phase:   0%|          | 0/939 [00:00<?, ?it/s]

Validation loop:   0%|          | 0/235 [00:00<?, ?it/s]

Epoch 1/5 - Train loss: 0.8672749136034472, Validation loss: 0.5158027584301129


Training Phase:   0%|          | 0/939 [00:00<?, ?it/s]